In [52]:
import sys
from pathlib import Path

sys.path.append("..")
from ribasim_lumping import RibasimLumpingNetwork

import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString

from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning
import warnings

warnings.simplefilter("ignore", category=NumbaDeprecationWarning)

In [53]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [54]:
# Define base_dir, results_dir and network name
base_dir = Path("..\\..\\ribasim_lumping_data\\")
results_dir = Path(base_dir, "results")
dhydro_dir = Path(base_dir, "d-hydro\\")
network_name = "zutphen_tki_netwerk"

In [55]:
simulation_name = 'tki_zuthpen_berkel_basis.dsproj_data'
file_bc = f'{base_dir}\\d-hydro\\{simulation_name}\\FlowFM\\input\\FlowFM_boundaryconditions1d.bc'

In [56]:
# Load areas (discharge units: afwaterende eenheden)
areas_file_path = Path(base_dir, "afw_eenheden\\wrij_afwateringseenheden_selectie_Zutphen.shp")
areas_gdf = gpd.read_file(areas_file_path)
areas_gdf = areas_gdf[['GFEIDENT', 'geometry']]

In [57]:
# Create networkanalysis
network = RibasimLumpingNetwork(
    name=network_name, 
    results_dir=results_dir,
    areas_gdf=areas_gdf,
)
# network.export_to_geopackage()

In [58]:
# Select simulation sets and extract all data using xugrid/ugrid
network.add_data_from_simulations_set(
    set_name="winter",
    simulations_dir=dhydro_dir,
    simulations_names=["tki_zuthpen_berkel_basis.dsproj"],
    simulations_ts=pd.date_range("2000-01-02 23:00", periods=9, freq="2D"),
);

 - Simulation set (winter): tki_zuthpen_berkel_basis.dsproj | Timestamps: 9 | his.nc and map.nc


In [59]:
# Read network data and extract all objects (weirs/pumps/laterals/confluences/bifurcations)
network.get_network_data(file_bc)

D-HYDRO-network analysed:
 - nodes and waterlevels
 - edges and discharges
 - stations / pumps / weirs / orifices / bridges / culverts / uniweirs / confluences / bifurcations / boundaries


In [60]:
# Define locations where the network should be split into Ribasim basins:
network.add_split_nodes(
    weirs=True,
    pumps=True,
    uniweirs=True,
    structures_ids_to_include=[
        'kdu_DR80760025', # duiker vispassage bovenstrooms
        'kst_ST80830001', 'kst_ST80810015', # onderdoorlaten bij verdeelpunt De Berkel (Zutphen)
        'kdu_DR84930010', # duiker met terugslagklep Zutphen Noorderhaven (parallel aan gemaal)
        'kdu_DR80950033', # duikers voor wijk Leesten
        'kdu_DR80940046', 'kdu_DR80950043', 'kdu_DR80950151', # duikers voor wijk Zuidwijken
        'kdu_DR80950103',
    ], 
    structures_ids_to_exclude=[
        'BCAL_3', 'BCAL_11', # stuwen voor hoogwaterafvoer De Berkel
        'BBypass_Besselink_1', 'BBypass_Besselink_2', 'BBypass_Besselink_3', 'BBypass_Besselink_4', 'BBypass_Besselink_5', # visdrempels vispassage De Berkel
        'kst_ST80950035', # verwarrende stuw ivm afwaterende eenheid (Zutphen: Leesten)
        'kst_ST84930001', # verwarrende stuw ivm afwaterende eenheid (Zutphen: Noorderhaven)
    ], 
    node_ids_to_include=[
        419, # voorbeeld splitsing
        1455, # extra punt rondom verdeelpunt De Berkel
    ],
    node_ids_to_exclude=[],
);

70 split locations
 - pump: 4
 - weir: 46
 - uniweir: 9
 - orifice: 2
 - culvert: 7
 - node: 2


In [61]:
# Create basins (gdf) based on nodes, edges, split_node_ids and areas
network.create_basins_and_connections_based_on_split_nodes();
# Export to geopackage
network.export_to_geopackage()

Create basins using split nodes:
 - create network graph
 - split network graph
 - define basins based on sub-networks
 - check if split locations are used
 - define basin areas
 - create final basins
 - create basin connections
 - create boundary-basin connections
Exporting to geopackage:
 - areas, nodes, edges, stations, pumps, weirs, orifices, bridges, culverts, uniweirs, basin_areas, split_nodes, basins, basin_connections, boundaries, boundary_basin_connections, 
 - not available: 
Export location: ..\..\ribasim_lumping_data\results\zutphen_tki_netwerk\ribasim_network.qgz:
